In [1]:
library(DBI)
library(dplyr)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [6]:
install.packages("RSQLite")
library(RSQLite)

also installing the dependency 'plogr'




package 'plogr' successfully unpacked and MD5 sums checked
package 'RSQLite' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\piorizzielloa\AppData\Local\Temp\2\RtmpGQDaMb\downloaded_packages


Warning message:
"package 'RSQLite' was built under R version 4.2.3"


In [3]:
getwd()

[1] "C:/Users/piorizzielloa/Documents/Data Science"

In [7]:
# part 2
# ======== create the database ========
if (file.exists("airline2.db")) 
  file.remove("airline2.db")
conn <- dbConnect(RSQLite::SQLite(), "airline2.db")

In [9]:
# ======== write to the database ========
# load in the data from the csv files
airports <- read.csv("airports.csv", header = TRUE)
carriers <- read.csv("carriers.csv", header = TRUE)
planes <- read.csv("plane-data.csv", header = TRUE)
dbWriteTable(conn, "airports", airports)
dbWriteTable(conn, "carriers", carriers)
dbWriteTable(conn, "planes", planes)

for(i in c(2006:2008)) {
  ontime <- read.csv(paste0(i, ".csv"), header = TRUE)
  if(i == 2006) {
    dbWriteTable(conn, "ontime", ontime)
  } else {
    dbWriteTable(conn, "ontime", ontime, append = TRUE)
  }
}

In [10]:
# ======== queries via DBI ========
q1 <- dbGetQuery(conn, 
                 "SELECT model AS model, AVG(ontime.DepDelay) AS avg_delay
FROM planes JOIN ontime USING(tailnum)
WHERE ontime.Cancelled = 0 AND ontime.Diverted = 0 AND ontime.DepDelay > 0
GROUP BY model
ORDER BY avg_delay")
print(paste(q1[1, "model"], "has the lowest associated average departure delay."))

[1] "737-230 has the lowest associated average departure delay."


In [11]:
q2 <- dbGetQuery(conn, 
                 "SELECT airports.city AS city, COUNT(*) AS total
FROM airports JOIN ontime ON ontime.dest = airports.iata
WHERE ontime.Cancelled = 0
GROUP BY airports.city
ORDER BY total DESC")
print(paste(q2[1, "city"], "has the highest number of inbound flights (excluding cancelled flights)"))

[1] "Chicago has the highest number of inbound flights (excluding cancelled flights)"


In [12]:
q3 <- dbGetQuery(conn, 
                 "SELECT carriers.Description AS carrier, COUNT(*) AS total
FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
WHERE ontime.Cancelled = 1
AND carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
GROUP BY carriers.Description
ORDER BY total DESC")

print(paste(q3[1, "carrier"], "has the highest number of cancelled flights"))

[1] "American Airlines Inc. has the highest number of cancelled flights"


In [13]:
#The CAST() function converts a value (of any type) into a specified datatype
q4 <- dbGetQuery(conn, 
                 "SELECT 
q1.carrier AS carrier, (CAST(q1.numerator AS FLOAT)/ CAST(q2.denominator AS FLOAT)) AS ratio
FROM
(
  SELECT carriers.Description AS carrier, COUNT(*) AS numerator
  FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
  WHERE ontime.Cancelled = 1 AND carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
  GROUP BY carriers.Description
) AS q1 JOIN 
(
  SELECT carriers.Description AS carrier, COUNT(*) AS denominator
  FROM carriers JOIN ontime ON ontime.UniqueCarrier = carriers.Code
  WHERE carriers.Description IN ('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')
  GROUP BY carriers.Description
) AS q2 USING(carrier)
ORDER BY ratio DESC")
print(paste(q4[1, "carrier"], "highest number of cancelled flights, relative to their number of total flights"))

[1] "Pinnacle Airlines Inc. highest number of cancelled flights, relative to their number of total flights"


In [14]:
# ======== queries via dplyr ========

planes_db <- tbl(conn, "planes")
ontime_db <- tbl(conn, "ontime")
carriers_db <- tbl(conn, "carriers")
airports_db <- tbl(conn, "airports")

In [15]:
q1 <- ontime_db %>% 
  rename_all(tolower) %>%
  inner_join(planes_db, by = "tailnum", suffix = c(".ontime", ".planes")) %>%
  filter(Cancelled == 0 & Diverted == 0 & DepDelay > 0) %>%
  group_by(model) %>%
  summarize(avg_delay = mean(DepDelay, na.rm = TRUE)) %>%
  arrange(avg_delay) 

print(head(q1, 1))

# Source:     SQL [1 x 2]
# Database:   sqlite 3.41.2 [C:\Users\piorizzielloa\Documents\Data Science\airline2.db]
# Ordered by: avg_delay
  model   avg_delay
  <chr>       <dbl>
1 737-230      13.0


In [16]:
#Use n() to Count Observations by Group
q2 <- ontime_db %>% 
  inner_join(airports_db, by = c("Dest" = "iata")) %>%
  filter(Cancelled == 0) %>%
  group_by(city) %>%
  summarize(total = n()) %>%
  arrange(desc(total)) 

print(head(q2, 1))

# Source:     SQL [1 x 2]
# Database:   sqlite 3.41.2 [C:\Users\piorizzielloa\Documents\Data Science\airline2.db]
# Ordered by: desc(total)
  city      total
  <chr>     <int>
1 Chicago 1051340


In [17]:
q3 <- ontime_db %>% 
  inner_join(carriers_db, by = c("UniqueCarrier" = "Code")) %>%
  filter(Cancelled == 1 & Description %in% c('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')) %>%
  group_by(Description) %>%
  summarize(total = n()) %>%
  arrange(desc(total))

print(head(q3, 1))


# Source:     SQL [1 x 2]
# Database:   sqlite 3.41.2 [C:\Users\piorizzielloa\Documents\Data Science\airline2.db]
# Ordered by: desc(total)
  Description            total
  <chr>                  <int>
1 American Airlines Inc. 38835


In [18]:
q4a <- inner_join(ontime_db, carriers_db, by = c("UniqueCarrier" = "Code")) %>%
  filter(Cancelled == 1 & Description %in% c('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')) %>%
  group_by(Description) %>%
  summarize(numerator = n()) %>%
  rename(carrier = Description)


In [19]:
q4b <- inner_join(ontime_db, carriers_db, by = c("UniqueCarrier" = "Code")) %>%
  filter(Description %in% c('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')) %>%
  group_by(Description) %>%
  summarize(denominator = n()) %>%
  rename(carrier = Description)

In [20]:
# mutate_if() – alterations all variables that satisfy a specific criterion.
# mutate() – adds new variables while retaining old variables to a data frame.

q4 <- inner_join(q4a, q4b, by = "carrier") %>%
  mutate_if(is.integer, as.double) %>%
  mutate(ratio = numerator/denominator) %>%
  select(carrier, ratio) %>%
  arrange(desc(ratio)) 

print(head(q4, 1))

Applying predicate on the first 100 rows


# Source:     SQL [1 x 2]
# Database:   sqlite 3.41.2 [C:\Users\piorizzielloa\Documents\Data Science\airline2.db]
# Ordered by: desc(ratio)
  carrier                 ratio
  <chr>                   <dbl>
1 Pinnacle Airlines Inc. 0.0347


In [21]:
# part 4: A simplified solution for the query 4 in practice quiz (G to H)
q4_simplified <- inner_join(ontime_db, carriers_db, by = c("UniqueCarrier" = "Code")) %>%
  filter(Description %in% c('United Air Lines Inc.', 'American Airlines Inc.', 'Pinnacle Airlines Inc.', 'Delta Air Lines Inc.')) %>%
  rename(carrier = Description) %>%
  group_by(carrier) %>%
  summarise(ratio = mean(Cancelled, na.rm = TRUE)) %>%
  arrange(desc(ratio))

print(head(q4_simplified, 1))

# Source:     SQL [1 x 2]
# Database:   sqlite 3.41.2 [C:\Users\piorizzielloa\Documents\Data Science\airline2.db]
# Ordered by: desc(ratio)
  carrier                 ratio
  <chr>                   <dbl>
1 Pinnacle Airlines Inc. 0.0347


In [23]:
dbDisconnect(conn)

Warning message in connection_release(conn@ptr):
"Already disconnected"
